In [25]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('Q:\Business Analytics\Data Science\ZipcodesGroupedColumns.csv')

In [26]:
raw_gzip = pd.read_csv(r'Q:\Business Analytics\Data Science\ZipcodesGroupedColumns.csv', index_col = 0)
new_sales = pd.read_csv(r'Q:\Business Analytics\Data Science\StorePOSDepartmentSalesByFY.csv')
new_sales.columns = [col.lower() for col in new_sales.columns.values]
gzip = new_sales.merge(raw_gzip[raw_gzip.columns.difference(['deptsales', 'fyyearnumber','departmentdescription','deptnumber'])], on='locationdimensionid')
gzip.drop_duplicates(inplace=True)
gzip.to_csv("newZipCodesGroupedColumns.csv", index=False)

In [27]:
uniqueDeptList = df.deptnumber.unique().tolist()

In [28]:
DeptCount = dict(pd.Series(df['deptnumber'].value_counts()))
DeptToDrop = []
for key in DeptCount:
    if DeptCount[key] < 400:
        DeptToDrop.append(key)
print(len(DeptToDrop))

for item in uniqueDeptList:
    if item in DeptToDrop:
        uniqueDeptList.remove(item)

24


In [29]:
to_float = {}
# This will change all columns that can be represented as floats to floats.
for rowname in df.columns.values:
    val = df[rowname][0]
    try:
        val = float(val)
    except ValueError:
        continue
    to_float[rowname] = float


# make all categoricla into numeric types using category codes from pandas
for col_name in df.columns:
    if(df[col_name].dtype == 'object'):
        df[col_name]= df[col_name].astype('category')
        df[col_name] = df[col_name].cat.codes

In [30]:
df.columns.values

array(['Unnamed: 0', 'totpop_cy', 'popdens_cy', 'medage_cy', 'divindx_cy',
       'low_income', 'medium_income', 'high_income', 'medhinc_cy',
       'medval_cy', 'inc_educ', 'hs_educ', 'coll_ed', 'grad_ed',
       'avghhsz_cy', 'avgfmsz_cy', 'tothh_cy', 'famhh_cy', 'blue_collar',
       'white_collar', 'unemployed', 'zipcode', 'locationdimensionid',
       'storenumber', 'storename', 'id', 'name', 'rg_name', 'rg_abbrev',
       'fyyearnumber', 'deptnumber', 'departmentdescription', 'deptsales'],
      dtype=object)

In [31]:
argstring = 'deptsales ~'
for colname in df.drop(columns=['Unnamed: 0', 'zipcode', 'locationdimensionid',
       'storenumber', 'storename', 'id', 'name', 'rg_name', 'rg_abbrev',
       'fyyearnumber', 'deptnumber', 'departmentdescription', 'deptsales']): # add additional column names to exclude them as features that are considered by the model
    argstring = argstring + (' ' + colname + ' +')
argstring = argstring[:-2]

In [34]:
import statsmodels.formula.api as smf
resultDict = {}
for uniqueDept in uniqueDeptList:
    DeptSubset = df[df['deptnumber'] == int(uniqueDept)]
    lm = smf.ols(formula='deptsales ~ totpop_cy + popdens_cy + medage_cy + divindx_cy + low_income + medium_income + high_income + medhinc_cy + medval_cy + inc_educ + hs_educ + coll_ed + grad_ed + avghhsz_cy + avgfmsz_cy + tothh_cy + famhh_cy + blue_collar + white_collar + unemployed', data=DeptSubset).fit()
    resultDict[int(uniqueDept)] = (lm.pvalues)
resultDict

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


{1: Intercept        7.243804e-02
 totpop_cy        5.342736e-02
 popdens_cy       5.279573e-05
 medage_cy        8.091442e-03
 divindx_cy       2.340813e-02
 low_income       4.052099e-07
 medium_income    1.068233e-01
 high_income      8.394574e-03
 medhinc_cy       2.626994e-03
 medval_cy        5.649865e-04
 inc_educ         1.286996e-02
 hs_educ          1.821935e-02
 coll_ed          3.718736e-01
 grad_ed          4.325160e-01
 avghhsz_cy       6.757220e-01
 avgfmsz_cy       2.471418e-01
 tothh_cy         3.708801e-01
 famhh_cy         2.796142e-03
 blue_collar      1.654627e-01
 white_collar     1.118160e-03
 unemployed       1.659414e-01
 dtype: float64, 2: Intercept        0.725010
 totpop_cy        0.163576
 popdens_cy       0.016491
 medage_cy        0.220711
 divindx_cy       0.006685
 low_income       0.008022
 medium_income    0.181180
 high_income      0.002074
 medhinc_cy       0.000144
 medval_cy        0.000051
 inc_educ         0.007882
 hs_educ          0.000265
 co

In [32]:
import statsmodels.formula.api as smf

DeptModelDict = {}

# run an ols model for each upc with > 50 observations
for uniqueDept in uniqueDeptList:
    lm = smf.ols(formula=argstring,data= df[df['deptnumber'].isin([uniqueDept])]).fit()
    tempdict = dict(lm.pvalues)
    finaldict = {}
    for item in tempdict:
        if tempdict[item] < 0.01:
            finaldict[item] = tempdict[item]
    if len(finaldict) > 1:
        DeptModelDict[uniqueDept] = finaldict

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [33]:
finaldict

{}